## Object recognition with YOLO ##


In [ ]:
import os
import torch
import cv2
import pandas as pd
from ultralytics import YOLO
import matplotlib.pyplot as plt


def get_folder_path(prompt, create_if_missing=False):
    while True:
        path = input(prompt).strip()
        if path:
            if os.path.exists(path) or create_if_missing:
                os.makedirs(path, exist_ok=True)  # Create folder if missing
                print(f"✅ Using folder: {path}")
                return path
        print("❌ Invalid or empty path. Please enter a valid folder path.")

# Function to list available folders before selection
def list_available_folders(base_path):
    print("📂 Available folders:")
    folders = [f for f in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, f))]
    if not folders:
        print("❌ No folders found. Exiting.")
        exit()
    for idx, folder in enumerate(folders, start=1):
        print(f"{idx}. {folder}")
    return folders

# Load YOLO model (using YOLOv8 pretrained model)
def load_yolo_model():
    print("🔄 Loading YOLOv8 model...")
    model = YOLO("yolov8n.pt") 
    print("✅ YOLO model loaded successfully!")
    return model

# Perform object detection on an image
def detect_objects(model, image_path):
    image = cv2.imread(image_path)
    results = model(image)  # Perform detection
    detections = []
    
    for result in results:
        for box in result.boxes.data:
            x1, y1, x2, y2, conf, cls = box.tolist()
            detections.append({
                "image": os.path.basename(image_path),
                "class": model.names[int(cls)],
                "confidence": round(conf, 2),
                "bbox": [int(x1), int(y1), int(x2), int(y2)]
            })
    return detections

# Visualize detections and save annotated images
def visualize_and_save(image_path, detections, output_folder):
    image = cv2.imread(image_path)
    for det in detections:
        x1, y1, x2, y2 = det["bbox"]
        label = f"{det['class']} ({det['confidence']})"
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    output_path = os.path.join(output_folder, os.path.basename(image_path))
    cv2.imwrite(output_path, image)
    print(f"✅ Saved annotated image: {output_path}")

# Function to manually select which movies to process
def select_movies(movies):
    print("📂 Available movie folders:")
    for idx, movie in enumerate(movies, start=1):
        print(f"{idx}. {movie}")
    
    selected_indices = input("Enter the numbers of the movies you want to analyze (comma-separated, e.g., 1,3,5): ")
    selected_indices = [int(i.strip()) - 1 for i in selected_indices.split(',') if i.strip().isdigit() and 0 <= int(i.strip()) - 1 < len(movies)]
    
    if not selected_indices:
        print("❌ No valid selections made. Exiting.")
        exit()
    
    selected_movies = [movies[i] for i in selected_indices]
    return selected_movies

# Function to process all selected movies
def process_selected_movies(model, keyframe_base_folder, output_image_folder):
    movies = list_available_folders(keyframe_base_folder)
    selected_movies = select_movies(movies)
    
    for movie in selected_movies:
        movie_path = os.path.join(keyframe_base_folder, movie)
        output_csv = os.path.join(keyframe_base_folder, f"object_detection_results_{movie}.csv")  # Unique CSV for each movie
        detections_list = []
        
        print(f"📂 Processing keyframes from movie: {movie}")
        keyframe_images = [f for f in os.listdir(movie_path) if f.endswith(".jpg")]
        for img in keyframe_images:
            img_path = os.path.join(movie_path, img)
            detections = detect_objects(model, img_path)
            detections_list.extend(detections)
            visualize_and_save(img_path, detections, output_image_folder)
        
        # Save detections to unique CSV for each movie
        df = pd.DataFrame(detections_list)
        df.to_csv(output_csv, index=False)
        print(f"✅ Object detection results saved to {output_csv}")


if __name__ == "__main__":
    keyframe_base_folder = get_folder_path("Enter the base folder containing movie keyframes: ")
    output_image_folder = get_folder_path("Enter the folder to save annotated images: ", create_if_missing=True)
    
    yolo_model = load_yolo_model()
    process_selected_movies(yolo_model, keyframe_base_folder, output_image_folder)
    
    print("✅ Object detection process completed!")



✅ Using folder: /Users/alicja/Desktop/VideoAnalyse/Corpus/KeyFrames/DramaKeyframes
✅ Using folder: /Users/alicja/Desktop/VideoAnalyse/Corpus/ObjectRecognition/DramaRecognition/Drama_05
🔄 Loading YOLOv8 model...
✅ YOLO model loaded successfully!
📂 Available folders:
1. Drama_04
2. Drama_03
3. Drama_02
4. Drama_05
5. Drama_01
📂 Available movie folders:
1. Drama_04
2. Drama_03
3. Drama_02
4. Drama_05
5. Drama_01
📂 Processing keyframes from movie: Drama_05

0: 416x640 1 person, 1 tie, 2 wine glasss, 70.0ms
Speed: 3.5ms preprocess, 70.0ms inference, 1.2ms postprocess per image at shape (1, 3, 416, 640)
✅ Saved annotated image: /Users/alicja/Desktop/VideoAnalyse/Corpus/ObjectRecognition/DramaRecognition/Drama_05/Drama_05_Мой ласковый и нежный зверь (FullHD, драма, реж_scene_35_keyframe.jpg

0: 416x640 1 person, 59.0ms
Speed: 2.2ms preprocess, 59.0ms inference, 0.7ms postprocess per image at shape (1, 3, 416, 640)
✅ Saved annotated image: /Users/alicja/Desktop/VideoAnalyse/Corpus/ObjectRec

## Merging all the CSV files for every Object Recognition done on each file ##


In [ ]:
import os
import pandas as pd



def get_folder_path(prompt):
    while True:
        path = input(prompt).strip()
        if path and os.path.exists(path):
            print(f"✅ Using folder: {path}")
            return path
        print("❌ Invalid or empty path. Please enter a valid folder path.")

csv_folder = get_folder_path("Enter the folder containing all object detection CSV files: ")
output_file = os.path.join(csv_folder, "merged_object_detection_results.csv")


csv_files = [f for f in os.listdir(csv_folder) if f.endswith(".csv")]

if not csv_files:
    print("❌ No CSV files found in the folder. Exiting.")
    exit()

print(f"📂 Found {len(csv_files)} CSV files. Merging...")

# Merge all CSV files into one dataframe
df_list = []
for csv_file in csv_files:
    file_path = os.path.join(csv_folder, csv_file)
    df = pd.read_csv(file_path)
    df["source_file"] = csv_file  # Add filename as a column
    df_list.append(df)

merged_df = pd.concat(df_list, ignore_index=True)
merged_df.to_csv(output_file, index=False)

print(f"✅ Merged dataset saved as: {output_file}")


✅ Using folder: /Users/alicja/Desktop/VideoAnalyse/ObjectRecognition_Results
📂 Found 15 CSV files. Merging...
✅ Merged dataset saved as: /Users/alicja/Desktop/VideoAnalyse/ObjectRecognition_Results/merged_object_detection_results.csv


## Cleaning the merged data ## 
Loads the merged dataset for analysis.

Displays basic information (column names, data types, missing values)

Removes duplicates to clean the data


Saves the cleaned dataset as cleaned_object_detection_results.csv



In [ ]:
import pandas as pd
import os



def get_file_path(prompt):
    while True:
        path = input(prompt).strip()
        if path and os.path.exists(path):
            print(f"✅ Using file: {path}")
            return path
        print("❌ Invalid or empty path. Please enter a valid file path.")

# Get the merged CSV file
csv_file = get_file_path("Enter the path to the merged object detection CSV file: ")


df = pd.read_csv(csv_file)

# Display basic info
print(" Dataset Overview:")
print(df.info())
print("\nSample Data:")
print(df.head())

# Check for missing values
missing_values = df.isnull().sum()
print("\n❗ Missing Values:")
print(missing_values[missing_values > 0])

# Remove duplicates (if any)
df = df.drop_duplicates()
print("✅ Duplicates removed.")

cleaned_file = os.path.join(os.path.dirname(csv_file), "cleaned_object_detection_results.csv")
df.to_csv(cleaned_file, index=False)
print(f"✅ Cleaned dataset saved as: {cleaned_file}")


✅ Using file: /Users/alicja/Desktop/VideoAnalyse/ObjectRecognition_Results/merged_object_detection_results.csv
📊 Dataset Overview:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19000 entries, 0 to 18999
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   image        19000 non-null  object 
 1   class        19000 non-null  object 
 2   confidence   19000 non-null  float64
 3   bbox         19000 non-null  object 
 4   source_file  19000 non-null  object 
dtypes: float64(1), object(4)
memory usage: 742.3+ KB
None

🔍 Sample Data:
                                               image       class  confidence  \
0  Drama_05_Мой ласковый и нежный зверь (FullH...      person        0.80   
1  Drama_05_Мой ласковый и нежный зверь (FullH...         tie        0.72   
2  Drama_05_Мой ласковый и нежный зверь (FullH...  wine glass        0.33   
3  Drama_05_Мой ласковый и нежный зверь (FullH...  wine glass        